## Imports

In [1]:
#importing necessary libraries
from api.dani_mapbox_api_walking import MapboxAPI
import requests
from urllib.request import urlopen
import json
from shapely.geometry import shape, Polygon
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('default')
import geopandas as gpd
import geodatasets
import ast
from shapely import wkt
from pyproj import Geod

## Importing Coverage Function (allows me to pick any branch I want)

In [2]:
branchInfo = pd.read_csv('../../can-summer-2023/notebooks/branchInfo.csv')

In [3]:
branchInfo

,Unnamed: 0,BRANCH,LOCATION,geometry,LATITUDE,LONGITUDE,list of dict
0,0,Albany Park,"(41.97557881655979, -87.71361314512697)","POLYGON ((-87.713613 41.990993, -87.714753 41....",41.975579,-87.713613,"[{17031140800: 0.24265316937897238, 1703104070..."
1,1,Altgeld,"(41.65719847558056, -87.59883829075173)","POLYGON ((-87.599838 41.666689, -87.60132 41.6...",41.657198,-87.598838,"[{17031540101: 0.2852409919978611, 17031550200..."
2,2,Archer Heights,"(41.80110836194246, -87.72648385568911)","POLYGON ((-87.723484 41.815393, -87.724484 41....",41.801108,-87.726484,"[{17031835200: 0.3111524694139818, 17031560300..."
3,3,Austin,"(41.88917312206642, -87.7658203582574)","POLYGON ((-87.76582 41.904379, -87.76782 41.90...",41.889173,-87.765820,"[{17031252101: 0.3298402493506001, 17031813000..."
4,4,Austin-Irving,"(41.95312267684315, -87.77928489355646)","POLYGON ((-87.776285 41.966283, -87.777285 41....",41.953123,-87.779285,"[{17031190401: 0.006368978144482472, 170311512..."
...,...,...,...,...,...,...,...
77,77,West Pullman,"(41.67790647382097, -87.6431683153105)","POLYGON ((-87.642168 41.692638, -87.645168 41....",41.677906,-87.643168,"[{17031821402: 0.06683673133829206, 1703182120..."
78,78,West Town,"(41.89594730478282, -87.66829305460979)","POLYGON ((-87.667293 41.910857, -87.668293 41....",41.895947,-87.668293,"[{17031838100: 0.06635765522527748, 1703183310..."
79,79,"Whitney M. Young, Jr.","(41.7510184591363, -87.61495065361933)","POLYGON ((-87.614951 41.766671, -87.616575 41....",41.751018,-87.614951,"[{17031440700: 0.02826420162848117, 1703144020..."
80,80,Woodson Regional Library,"(41.720694885749005, -87.64304817213312)","POLYGON ((-87.643048 41.73608, -87.644048 41.7...",41.720695,-87.643048,"[{17031490500: 0.000460208436123569, 170314911..."


In [4]:
branchInfo = branchInfo.drop(columns=['Unnamed: 0'])

In [5]:
branchInfo

,BRANCH,LOCATION,geometry,LATITUDE,LONGITUDE,list of dict
0,Albany Park,"(41.97557881655979, -87.71361314512697)","POLYGON ((-87.713613 41.990993, -87.714753 41....",41.975579,-87.713613,"[{17031140800: 0.24265316937897238, 1703104070..."
1,Altgeld,"(41.65719847558056, -87.59883829075173)","POLYGON ((-87.599838 41.666689, -87.60132 41.6...",41.657198,-87.598838,"[{17031540101: 0.2852409919978611, 17031550200..."
2,Archer Heights,"(41.80110836194246, -87.72648385568911)","POLYGON ((-87.723484 41.815393, -87.724484 41....",41.801108,-87.726484,"[{17031835200: 0.3111524694139818, 17031560300..."
3,Austin,"(41.88917312206642, -87.7658203582574)","POLYGON ((-87.76582 41.904379, -87.76782 41.90...",41.889173,-87.765820,"[{17031252101: 0.3298402493506001, 17031813000..."
4,Austin-Irving,"(41.95312267684315, -87.77928489355646)","POLYGON ((-87.776285 41.966283, -87.777285 41....",41.953123,-87.779285,"[{17031190401: 0.006368978144482472, 170311512..."
...,...,...,...,...,...,...
77,West Pullman,"(41.67790647382097, -87.6431683153105)","POLYGON ((-87.642168 41.692638, -87.645168 41....",41.677906,-87.643168,"[{17031821402: 0.06683673133829206, 1703182120..."
78,West Town,"(41.89594730478282, -87.66829305460979)","POLYGON ((-87.667293 41.910857, -87.668293 41....",41.895947,-87.668293,"[{17031838100: 0.06635765522527748, 1703183310..."
79,"Whitney M. Young, Jr.","(41.7510184591363, -87.61495065361933)","POLYGON ((-87.614951 41.766671, -87.616575 41....",41.751018,-87.614951,"[{17031440700: 0.02826420162848117, 1703144020..."
80,Woodson Regional Library,"(41.720694885749005, -87.64304817213312)","POLYGON ((-87.643048 41.73608, -87.644048 41.7...",41.720695,-87.643048,"[{17031490500: 0.000460208436123569, 170314911..."


In [14]:
altgeld_dictlist = branchInfo.loc[branchInfo['BRANCH'] == 'Altgeld','list of dict']
altgeld_dictlist
dl = altgeld_dictlist.tolist()
dl
dictionary = dl[0]
dictionary
#dictionary[7]

'[{17031540101: 0.2852409919978611, 17031550200: 0.006462013886188789, 17031540102: 0.23548056303373108, 17031838800: 0.0012776283041793645}]'